In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pygco import cut_simple
import os
import glob

In [2]:
115,  18, 254

(115, 18, 254)

In [3]:
def getCenterLabel(pixel):
    """
    background = black = (0,0,0)
    outside = red = (255,0,0)
    inside = green = (255,255,0)
    lumen = yellow = (0,255,0)
    catheder shadow = blue = (0,0,255)
    artery wall = turquisw = (0,255,255)
    stent = pink = (255,0,255)
    
    input: RGB value of pixel
    output: integer between 1-5 represnting a class described above
    """
    pixel = tuple(pixel)
    rgb2label = {(115,  18, 254):0, (255,255,0):1, (0,255,0):2, 
                    (0,0,255):3, (0,255,255):4, (255,0,255):5}
    return rgb2label.get(pixel, 6)


In [10]:
def pot_penalizing(unaries):
    # potts potential
    label_num = unaries.shape[2]
    x = np.argmin(unaries, axis=2)
    # potential that penalizes 0-1 and 1-2 less thann 0-2
    A = -1000
    B = 800
    pairwise_1d = A * np.eye(label_num, dtype=np.int32) - B
    pairwise_1d[3, 5] = 0
    pairwise_1d[5, 3] = 0
    pairwise_1d[0, 5] = 0
    pairwise_1d[5, 0] = 0
    pairwise_1d[1, 5] = 0
    pairwise_1d[5, 1] = 0
    pairwise_1d[0, 3] = 0
    pairwise_1d[3, 0] = 0
    pairwise_1d[1, 4] = 0
    pairwise_1d[4, 1] = 0
    pairwise_1d[0, 1] = 0
    pairwise_1d[1, 0] = 0
    pairwise_1d[2, 1] = -B/2
    pairwise_1d[1, 2] = -B/2
    pairwise_1d
    print(pairwise_1d)
    result = cut_simple(unaries, pairwise_1d)
    plt.subplot(131, title="original")
    plt.imshow(x, interpolation="nearest")
    plt.subplot(133, title="potts potentials penalizing")
    plt.imshow(result, interpolation="nearest")
    plt.show()
    

In [5]:
def makevector(labels, num_labels):
    labels = np.asarray(labels)
    return (np.arange(num_labels) == labels[:,None]).astype(np.float32)

In [6]:
def vectorized_result(j,num_label):
    """Return a 5-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((num_label, 1))
    e[int(j)] = 1
    return e

In [7]:
cwd_eval = '/Users/jbaldauf/Documents/Tensorflow/OCT-project/Data/first_results/'
img1 = np.asarray(Image.open(cwd_eval + '1Image064_zoom.png'))
img1 = img1.reshape(-1,3)
num_labels = 7
img1 = [vectorized_result(getCenterLabel(img1[i]),7) for i in xrange(len(img1))]
img1 = np.asarray(img1)
unaries = (img1.reshape(900,900,7)*(-100)).astype(np.int32)
#unaries = (unaries + 0.5 * np.random.normal(size=unaries.shape)).astype(np.int32)

In [8]:
def smoothing_oct(unaries):
    #number of labels:
    n_disps = unaries.shape[2]
    #shape of the input image
    newshape = unaries.shape[:2]
    
    #print newshape
    #array([[-5,  0,  0,  0,  0,  0],
    #   [ 0, -5,  0,  0,  0,  0],
    #   [ 0,  0, -5,  0,  0,  0],
    #   [ 0,  0,  0, -5,  0,  0],
    #   [ 0,  0,  0,  0, -5,  0],
    #   [ 0,  0,  0,  0,  0, -5]], dtype=int32)
    potts_cut = cut_simple(unaries, -1000 * np.eye(n_disps, dtype=np.int32), algorithm = 'expansion')
    
    # x  = array([[0],    y = array([[0, 1, 2, 3, 4, 5]])
    #   [1],
    #   [2],
    #   [3],
    #   [4],
    #   [5]])
    x, y = np.ogrid[:n_disps, :n_disps]
    
    #create an array
    #array([[0, 1, 2, 3, 4, 5],
    #   [1, 0, 1, 2, 3, 4],
    #   [2, 1, 0, 1, 2, 3],
    #   [3, 2, 1, 0, 1, 2],
    #   [4, 3, 2, 1, 0, 1],
    #   [5, 4, 3, 2, 1, 0]])
    one_d_topology = np.abs(x - y).astype(np.int32).copy("C")
    one_d_cut = cut_simple(unaries, 300 * one_d_topology)
    
    #return potts_cut.reshape(newshape)
    plt.subplot(131, xticks=(), yticks=())
    plt.imshow(np.argmin(unaries, axis=2), interpolation='nearest')
    plt.subplot(132, xticks=(), yticks=())
    plt.imshow(potts_cut.reshape(newshape), interpolation='nearest')
    plt.subplot(133, xticks=(), yticks=())
    plt.imshow(one_d_cut.reshape(newshape), interpolation='nearest')
    plt.show()

In [ ]:
test = pot_penalizing(unaries)


[[-1800     0  -800     0  -800     0  -800]
 [    0 -1800  -400  -800     0     0  -800]
 [ -800  -400 -1800  -800  -800  -800  -800]
 [    0  -800  -800 -1800  -800     0  -800]
 [ -800     0  -800  -800 -1800  -800  -800]
 [    0     0  -800     0  -800 -1800  -800]
 [ -800  -800  -800  -800  -800  -800 -1800]]


In [ ]:
cwd_raw = '/Users/jbaldauf/Documents/Tensorflow/OCT-project/Data/Trainingdata/Raw/'
cwd_label = '/Users/jbaldauf/Documents/Tensorflow/OCT-project/Data/Trainingdata/Multiple/'
cwd_save = '/Users/jbaldauf/Documents/Tensorflow/OCT-project/Data'

box_size = 12

data_train = sorted(glob.glob1(cwd_save,'*'))
print data_train

In [13]:
smoothing_oct(unaries)


In [14]:
test.shape


(900, 900)